# Resilient Distributed Datasets (RDD)

## Create a RDD

In [1]:
num_rdd = sc.parallelize(list(range(10000)))

In [2]:
type(num_rdd)

pyspark.rdd.RDD

## Read a RDD from HDFS

In [4]:
data_rdd = sc.textFile('hdfs://localhost/data.csv')

In [5]:
type(data_rdd)

pyspark.rdd.RDD

## Count

In [10]:
num_rdd.count()

10000

## Take

In [23]:
num_rdd.take(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Map, filter, reduce

In [3]:
num_rdd\
    .map(lambda x: x * x)\
    .filter(lambda x: x % 2 == 0)\
    .reduce(lambda a, b: a + b)

166616670000

## Map, filter

In [24]:
data_rdd\
    .filter(lambda s: False if s.startswith('x') else True)\
    .map(lambda s: s.split(','))\
    .map(lambda arr: [int(s) for s in arr])\
    .take(10)

[[73, 18, 90, 76, 23, 43, 8, 12, 69, 12],
 [37, 49, 11, 12, 25, 87, 71, 15, 42, 1],
 [89, 48, 29, 31, 6, 35, 83, 19, 54, 50],
 [69, 61, 24, 93, 81, 67, 93, 52, 49, 55],
 [10, 71, 71, 67, 23, 4, 45, 70, 49, 83],
 [93, 95, 24, 70, 77, 82, 2, 57, 52, 84],
 [95, 33, 49, 42, 16, 11, 85, 38, 62, 12],
 [58, 64, 24, 94, 72, 59, 86, 96, 4, 62],
 [25, 75, 72, 31, 6, 53, 45, 61, 41, 51],
 [15, 46, 89, 69, 38, 77, 17, 28, 29, 2]]